In [2]:
from tqdm import tqdm
import re
import pandas as pd

In [3]:
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

In [7]:
with open('./access.log') as source:
    line_num = 0
    arr = []
    
    for line in tqdm(source):
        new_line = None
        combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
        try:
            new_line = re.findall(combined_regex, line)[0]
            arr.append(new_line)
        except Exception as e:
            with open('errors_file.log', 'a') as errfile:
                print((type(line), str(e)), file=errfile)

        line_num += 1
        if line_num % 250_000 == 0:
            df = pd.DataFrame(arr, columns=columns)
            df.to_parquet(f'./data/{line_num}.parquet')
            arr.clear()

    df = pd.DataFrame(arr, columns=columns)
    df.to_parquet(f'./data/{line_num}.parquet')
    arr.clear()    

10365152it [02:36, 66119.99it/s] 
